In [40]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [41]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", chrome_options=options)

<ipython-input-41-04beeb22e94e>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", chrome_options=options)
<ipython-input-41-04beeb22e94e>:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", chrome_options=options)


In [42]:
URL = 'https://shotqualitybets.com'
LOGIN_ROUTE = '/login#login'


HEADERS = {
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
    , 'origin': URL
    , 'referer': URL + LOGIN_ROUTE
}

s = requests.session()

# csrf_token = s.get(URL).cookies['accesstoken']
login_payload = {
        'login': '',
        'password': ''
        # 'csrfmiddlewaretoken': csrf_token
        }

login_req = s.post(URL + LOGIN_ROUTE, headers=HEADERS, data=login_payload)
print(login_req.status_code)

200


In [43]:
model_results = '/model-results'
driver.get(URL + model_results)
driver.set_page_load_timeout(10)

In [44]:
## CREATE COLUMNS AND INITIATE DATA FRAMES
cols = [
    'date', 
    'home_team', 
    'away_team', 
    'projected_score', 
    'projected_total', 
    'consensus_ou', 
    'side_w_line_value', 
    'points_of_value', 
    'final_score', 
    'sq_score', 
    'play', 
]

ou_results = pd.DataFrame(columns=cols)
spread_results = pd.DataFrame(columns=cols)

In [45]:
## FIND OVER/UNDER DATA

btn = driver.find_element(By.XPATH, '//*[@id="ContentBox"]/div/section[1]/div[2]/div/div[4]/button[1]')
driver.execute_script("arguments[0].click();", btn)

for i in np.arange(1,240):
    # find day button to click and click
    btn = driver.find_element(By.XPATH, '//*[@id="ContentBox"]/div/section[2]/div/div[1]/div[2]/div/div[1]/div[2]/div/div/div/div[' + str(i) + ']/div/button')
    driver.execute_script("arguments[0].click();", btn)
    # get html for page after click
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # find the table with the data
    table = soup.find('table', class_='table')
    
    # look through table for data an add to data frame
    for row in table.tbody.find_all('tr'):
        c = row.find_all('td')
        if c != []:
            c = row.find_all('td')
            if c[0].text.strip() != 'There are no games available for this day. Try selecting another date.':
                date = c[0].text.strip()
                home_team = c[1].text.strip()
                away_team = c[2].text.strip()
                projected_score = c[3].text.strip()
                projected_total = c[4].text.strip()
                consensus_ou = c[5].text.strip()
                side_w_line_value = c[6].text.strip()
                points_of_value = c[7].text.strip()
                final_score = c[8].text.strip()
                sq_score = c[9].text.strip()
                play = c[10].text.strip()

                ou_results = ou_results.append(pd.DataFrame(
                    [[date, home_team, away_team, projected_score, projected_total
                    , consensus_ou, side_w_line_value, points_of_value, final_score, sq_score, play]] 
                    , columns=cols
                ))
    

In [46]:
## FIND SPREAD DATA

btn = driver.find_element(By.XPATH, '//*[@id="ContentBox"]/div/section[1]/div[2]/div/div[4]/button[2]')
driver.execute_script("arguments[0].click();", btn)

for i in np.arange(1,240):
    # find day button to click and click
    btn = driver.find_element(By.XPATH, '//*[@id="ContentBox"]/div/section[2]/div/div[1]/div[2]/div/div[1]/div[2]/div/div/div/div[' + str(i) + ']/div/button')
    driver.execute_script("arguments[0].click();", btn)
    # get html for page after click
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # find the table with the data
    table = soup.find('table', class_='table')
    
    # look through table for data an add to data frame
    for row in table.tbody.find_all('tr'):
        c = row.find_all('td')
        if c != []:
            c = row.find_all('td')
            if c[0].text.strip() != 'There are no games available for this day. Try selecting another date.':
                date = c[0].text.strip()
                home_team = c[1].text.strip()
                away_team = c[2].text.strip()
                projected_score = c[3].text.strip()
                projected_total = c[4].text.strip()
                consensus_ou = c[5].text.strip()
                side_w_line_value = c[6].text.strip()
                points_of_value = c[7].text.strip()
                final_score = c[8].text.strip()
                sq_score = c[9].text.strip()
                play = c[10].text.strip()

                spread_results = spread_results.append(pd.DataFrame(
                    [[date, home_team, away_team, projected_score, projected_total
                    , consensus_ou, side_w_line_value, points_of_value, final_score, sq_score, play]] 
                    , columns=cols
                ))

In [47]:
spread_results['home_team_score'] = spread_results['final_score'].str[:2]
spread_results['away_team_score'] = spread_results['final_score'].str[-2:]
spread_results

,date,home_team,away_team,projected_score,projected_total,consensus_ou,side_w_line_value,points_of_value,final_score,sq_score,play,home_team_score,away_team_score
0,2022-11-08,Georgetown,Coppin St.,87 - 71,-16.1,-18.5,Coppin St.,2.4,99 - 89,Only available to ShotQuality premiumTo see th...,,99,89
0,2022-11-08,Duquesne,Montana,66 - 67,1.4,-5.5,Montana,6.9,91 - 63,Only available to ShotQuality premiumTo see th...,+5.5,91,63
0,2022-11-08,East Carolina,Mercer,68 - 71,2.8,1,Mercer,1.8,77 - 75,Only available to ShotQuality premiumTo see th...,,77,75
0,2022-11-08,San Jose St.,Georgia Southern,64 - 67,2.8,-4,Georgia Southern,6.8,63 - 48,Only available to ShotQuality premiumTo see th...,+4,63,48
0,2022-11-08,Hartford,Sacred Heart,70 - 73,2.4,6.5,Hartford,4.1,70 - 77,Only available to ShotQuality premiumTo see th...,+6.5,70,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2023-02-27,South Carolina St.,Howard,69 - 78,8.9,5.5,Howard,3.4,81 - 82,Only available to ShotQuality premiumTo see th...,,81,82
0,2023-02-27,Iowa St.,West Virginia,68 - 67,-1.8,-4,West Virginia,2.2,69 - 72,Only available to ShotQuality premiumTo see th...,,69,72
0,2023-02-27,Oklahoma St.,Baylor,67 - 70,3.6,1.5,Baylor,2.1,68 - 74,Only available to ShotQuality premiumTo see th...,,68,74
0,2023-02-27,Jackson St.,Prairie View A&M,70 - 70,0.7,0,Prairie View A&M,0.7,61 - 57,Only available to ShotQuality premiumTo see th...,,61,57


In [48]:
ou_results['home_team_score'] = ou_results['final_score'].str[:2].astype(int)
ou_results['away_team_score'] = ou_results['final_score'].str[-2:].astype(int)
ou_results['final_total'] = ou_results['home_team_score'] + ou_results['away_team_score']
ou_results['ou_result'] = ou_results['final_total'] - ou_results['consensus_ou'].astype(float)
ou_results['ou_result'] = ou_results['ou_result'].apply(lambda x: 'Over' if x > 0 else 'Under' if x < 0 else 'Push')
ou_results['prediction_result'] = ou_results['side_w_line_value'] == ou_results['ou_result']
ou_results['prediction_result'] = ou_results['prediction_result'].apply(lambda x: 'Win' if x else 'Loss')